# Spark SQL and DataFrames

### 1. Basic manipulations
First, let's create some users:

In [0]:
from datetime import date

# Let's first create some users:
col_names = ["first_name", "last_name", "birth_date", "gender", "country"]
users = [
  ("Alice", "Jones", date(1981, 4, 15), "female", "Canada"),
  ("John", "Doe", date(1951, 1, 21), "male", "USA"),
  ("Barbara", "May", date(1951, 9, 1), "female", "Australia"),
  ("James", "Smith", date(1975, 7, 12), "male", "United Kingdom"),
  ("Gerrard", "Dupont", date(1968, 5, 9), "male", "France"),
  ("Amanda", "B.", date(1988, 12, 16), "female", "New Zeland")
]

users_df = spark.createDataFrame(users, col_names)
display(users_df) # Only works in Databricks. Elswehere, use "df.show()" or "df.toPandas()"

first_name,last_name,birth_date,gender,country
Alice,Jones,1981-04-15,female,Canada
John,Doe,1951-01-21,male,USA
Barbara,May,1951-09-01,female,Australia
James,Smith,1975-07-12,male,United Kingdom
Gerrard,Dupont,1968-05-09,male,France
Amanda,B.,1988-12-16,female,New Zeland


Now it's your turn, create more users (at least 3, with different names) and add to the initial users, saving the result in a new variable.

In [0]:
userTmp = [
  ("Aminata", "Camara", date(1986, 3, 14), "female", "Guinee"),
  ("Fatim", "Diarra", date(1945, 5, 12), "male", "Mali"),
  ("Moussa", "Keita", date(1958, 9, 6), "female", "Niger"),
  ("Ousmane", "Barry", date(1965, 4, 13), "male", "Senegal"),
]

new_users_df = spark.createDataFrame(userTmp, col_names)
all_users_df = users_df.union(new_users_df)
display(all_users_df) # or all_users_df.show()

first_name,last_name,birth_date,gender,country
Alice,Jones,1981-04-15,female,Canada
John,Doe,1951-01-21,male,USA
Barbara,May,1951-09-01,female,Australia
James,Smith,1975-07-12,male,United Kingdom
Gerrard,Dupont,1968-05-09,male,France
Amanda,B.,1988-12-16,female,New Zeland
Aminata,Camara,1986-03-14,female,Guinee
Fatim,Diarra,1945-05-12,male,Mali
Moussa,Keita,1958-09-06,female,Niger
Ousmane,Barry,1965-04-13,male,Senegal


Now, select only two columns and show the resulting DataFrame, without saving it into a variable.

In [0]:
all_users_df.select("first_name", "birth_date").show()

+----------+----------+
first_name|birth_date|
+----------+----------+
 Alice|1981-04-15|
 John|1951-01-21|
 Barbara|1951-09-01|
 James|1975-07-12|
 Gerrard|1968-05-09|
 Amanda|1988-12-16|
 Aminata|1986-03-14|
 Fatim|1945-05-12|
 Moussa|1958-09-06|
 Ousmane|1965-04-13|
+----------+----------+

Now, register your DataFrame as a table and select the same two columns with a SQL query string

In [0]:
query_string = "SELECT first_name, birth_date FROM prem_table"
all_users_df.createOrReplaceTempView("prem_table") # creates a local temporary table accessible by a SQL query
spark.sql(query_string).show()

+----------+----------+
first_name|birth_date|
+----------+----------+
 Alice|1981-04-15|
 John|1951-01-21|
 Barbara|1951-09-01|
 James|1975-07-12|
 Gerrard|1968-05-09|
 Amanda|1988-12-16|
 Aminata|1986-03-14|
 Fatim|1945-05-12|
 Moussa|1958-09-06|
 Ousmane|1965-04-13|
+----------+----------+

Now we want to add an unique identifier for each user in the table. There are many strategies for that, and for our example we will use the string `{last_name}_{first_name}`

You can use `all_users_df` since your latest operation did not override its values. Add a new column called `user_id` to your DataFrame and save to a new variable.

**Hint:** The first place to look is in the [functions](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions) package

In [0]:
from pyspark.sql import functions as fn

## On povait aussi rajouter fn.lower pour convertir les strings en miniscule
users_with_id = all_users_df.select(fn.concat_ws('_', all_users_df.last_name, all_users_df.first_name).alias('user_id'), "*")
display(users_with_id)

user_id,first_name,last_name,birth_date,gender,country
Jones_Alice,Alice,Jones,1981-04-15,female,Canada
Doe_John,John,Doe,1951-01-21,male,USA
May_Barbara,Barbara,May,1951-09-01,female,Australia
Smith_James,James,Smith,1975-07-12,male,United Kingdom
Dupont_Gerrard,Gerrard,Dupont,1968-05-09,male,France
B._Amanda,Amanda,B.,1988-12-16,female,New Zeland
Camara_Aminata,Aminata,Camara,1986-03-14,female,Guinee
Diarra_Fatim,Fatim,Diarra,1945-05-12,male,Mali
Keita_Moussa,Moussa,Keita,1958-09-06,female,Niger
Barry_Ousmane,Ousmane,Barry,1965-04-13,male,Senegal


You can also do the same thing with an User Defined Function by passing a lambda, although it is not recommended when there is already a function in the `functions` package.

Add a new column called `user_id_udf` to the DataFrame, using an UDF that receives two parameters and concatenate them.

In [0]:
concat_udf = fn.udf(lambda str1, str2: str1 + "_" + str2)
users_with_id_udf = all_users_df.withColumn("user_id_udf", concat_udf(all_users_df.last_name, all_users_df.first_name))
display(users_with_id_udf)

first_name,last_name,birth_date,gender,country,user_id_udf
Alice,Jones,1981-04-15,female,Canada,Jones_Alice
John,Doe,1951-01-21,male,USA,Doe_John
Barbara,May,1951-09-01,female,Australia,May_Barbara
James,Smith,1975-07-12,male,United Kingdom,Smith_James
Gerrard,Dupont,1968-05-09,male,France,Dupont_Gerrard
Amanda,B.,1988-12-16,female,New Zeland,B._Amanda
Aminata,Camara,1986-03-14,female,Guinee,Camara_Aminata
Fatim,Diarra,1945-05-12,male,Mali,Diarra_Fatim
Moussa,Keita,1958-09-06,female,Niger,Keita_Moussa
Ousmane,Barry,1965-04-13,male,Senegal,Barry_Ousmane


Now, let's add another column called `age` with the computed age (in years) of each user, based on a given reference date, and save the resulting DataFrame into a new variable.

**Hint:** You can first compute the age in months, and then divide by 12. A final operation will probably be needed to get an integer number.

In [0]:
reference_date = date(2017, 12, 31)

users_with_age = all_users_df.withColumn("age", fn.round(fn.months_between(fn.lit(reference_date),all_users_df.birth_date, True)/(12)))
display(users_with_age)

first_name,last_name,birth_date,gender,country,age
Alice,Jones,1981-04-15,female,Canada,37.0
John,Doe,1951-01-21,male,USA,67.0
Barbara,May,1951-09-01,female,Australia,66.0
James,Smith,1975-07-12,male,United Kingdom,42.0
Gerrard,Dupont,1968-05-09,male,France,50.0
Amanda,B.,1988-12-16,female,New Zeland,29.0
Aminata,Camara,1986-03-14,female,Guinee,32.0
Fatim,Diarra,1945-05-12,male,Mali,73.0
Moussa,Keita,1958-09-06,female,Niger,59.0
Ousmane,Barry,1965-04-13,male,Senegal,53.0


Now, an analytical question: How many users of each gender who are more than 40 years old exist in this data? The solution must be a DataFrame with two columns: `gender` and `count` and two lines, one for each gender.

Bonus: Try to do your solution in a single chain (without intermediate variables)

**Hint:** You will need to filter and aggregate the data.

In [0]:
result = users_with_age.select("*").where("age > 40").groupBy('gender').count()
display(result)

gender,count
male,5
female,2


### 2. Reading files, performing joins, and aggregating

For this section you will use some fake data of two datasets: `Users` and `Donations`. The data is provided in two CSV files *with header* and using *comma* as separator.

The `Users` dataset contains information about about the users. 

The `Donations` dataset contains information about Donations performed by those users.

The first task is to read these files into the appropriate DataFrames.

**Note:** You need to set the option "inferSchema" to true in order to have the columns in the correct types. Besides, you can use `DataFrame.withColumn` in combination with `fn.to_date` or `fn.to_timestamp` to get proper dates.

_The data for this section has been created using [Mockaroo](https://www.mockaroo.com/)_.

In [0]:
users_from_file = spark.read.csv("/FileStore/tables/users.csv", sep=",", header=True, inferSchema=True)
users_from_file = users_from_file.withColumn("birth_date", fn.to_date(users_from_file.birth_date)) 
display(users_from_file)

donations = spark.read.csv("/FileStore/tables/donations.csv", sep=",", header=True, inferSchema=True)
donations = donations.withColumn("date", fn.to_date(donations.date)) \
                     .withColumn("timestamp", fn.to_timestamp(donations.timestamp))
display(donations)

user_id,first_name,last_name,gender,birth_date,country
1,Janifer,Lagadu,Female,1940-08-09,United States
2,Bruis,Danilov,Male,1966-05-28,Brazil
3,Omar,Spinozzi,Male,1977-08-28,China
4,Mag,Kennedy,Female,1963-11-11,China
5,Gerome,Lushey,Male,1990-06-08,Brazil
6,Lazarus,Andrus,Male,1983-04-22,France
7,Adora,Bartunek,Female,1971-06-04,France
8,Gladys,Gulland,Female,1959-07-07,Brazil
9,Pearline,Kitcher,Female,1944-06-01,China
10,Jolie,Diggin,Female,1944-07-31,Nigeria


donation_id,user_id,donation_value,date,time,timestamp
1,63,25.41,2017-03-06,18:28:20,2017-03-06T18:28:20.000+0000
2,53,93.32,2017-03-15,15:15:09,2017-03-15T15:15:09.000+0000
3,74,136.04,2017-10-29,20:36:23,2017-10-29T20:36:23.000+0000
4,94,37.25,2017-05-16,12:33:58,2017-05-16T12:33:58.000+0000
5,83,89.45,2017-05-09,22:45:44,2017-05-09T22:45:44.000+0000
6,80,163.67,2017-07-25,21:50:36,2017-07-25T21:50:36.000+0000
7,19,10.69,2017-11-15,8:43:54,2017-11-15T08:43:54.000+0000
8,53,135.09,2017-04-14,7:43:37,2017-04-14T07:43:37.000+0000
9,46,94.38,2017-03-07,10:30:34,2017-03-07T10:30:34.000+0000
10,90,68.63,2017-02-02,14:23:43,2017-02-02T14:23:43.000+0000


Now investigate the columns, contents and size of both datasets

In [0]:
# print the column names and types
users_from_file.printSchema()
donations.printSchema()

# print 5 elements of the datasets in a tabular format
users_from_file.show(5)
donations.show(5)

# print the number of lines of each dataset
print(users_from_file.count())
print(donations.count())

root
-- user_id: integer (nullable = true)
-- first_name: string (nullable = true)
-- last_name: string (nullable = true)
-- gender: string (nullable = true)
-- birth_date: date (nullable = true)
-- country: string (nullable = true)

root
-- donation_id: integer (nullable = true)
-- user_id: integer (nullable = true)
-- donation_value: double (nullable = true)
-- date: date (nullable = true)
-- time: string (nullable = true)
-- timestamp: timestamp (nullable = true)

+-------+----------+---------+------+----------+-------------+
user_id|first_name|last_name|gender|birth_date| country|
+-------+----------+---------+------+----------+-------------+
 1| Janifer| Lagadu|Female|1940-08-09|United States|
 2| Bruis| Danilov| Male|1966-05-28| Brazil|
 3| Omar| Spinozzi| Male|1977-08-28| China|
 4| Mag| Kennedy|Female|1963-11-11| China|
 5| Gerome| Lushey| Male|1990-06-08| Brazil|
+-------+----------+---------+------+----------+-------------+
only showing top 5 rows

+-----------+-------+--------------+----------+--------+-------------------+
donation_id|user_id|donation_value| date| time| timestamp|
+-----------+-------+--------------+----------+--------+-------------------+
 1| 63| 25.41|2017-03-06|18:28:20|2017-03-06 18:28:20|
 2| 53| 93.32|2017-03-15|15:15:09|2017-03-15 15:15:09|
 3| 74| 136.04|2017-10-29|20:36:23|2017-10-29 20:36:23|
 4| 94| 37.25|2017-05-16|12:33:58|2017-05-16 12:33:58|
 5| 83| 89.45|2017-05-09|22:45:44|2017-05-09 22:45:44|
+-----------+-------+--------------+----------+--------+-------------------+
only showing top 5 rows

100
1000

**Note:** If all the column types shown in the previous results are "string", you need to make sure you passed "inferSchema" as true when reading the CSV files and that you have converted date and timestamp columns to the matching types before continuing.

Before using the data, we may want to add some information about the users. 

Add a column containing the age of each user.

##### Je garde toujours la date référence de la question similaire précédente

In [0]:
# Je garde toujours la référence date de la question similaire précédente

users_from_file = users_from_file.withColumn("age", fn.round(fn.months_between(fn.lit(reference_date),users_from_file.birth_date, True)/(12)))
users_from_file.show()
users_from_file.printSchema()

+-------+----------+----------+------+----------+-------------+----+
user_id|first_name| last_name|gender|birth_date| country| age|
+-------+----------+----------+------+----------+-------------+----+
 1| Janifer| Lagadu|Female|1940-08-09|United States|77.0|
 2| Bruis| Danilov| Male|1966-05-28| Brazil|52.0|
 3| Omar| Spinozzi| Male|1977-08-28| China|40.0|
 4| Mag| Kennedy|Female|1963-11-11| China|54.0|
 5| Gerome| Lushey| Male|1990-06-08| Brazil|28.0|
 6| Lazarus| Andrus| Male|1983-04-22| France|35.0|
 7| Adora| Bartunek|Female|1971-06-04| France|47.0|
 8| Gladys| Gulland|Female|1959-07-07| Brazil|58.0|
 9| Pearline| Kitcher|Female|1944-06-01| China|74.0|
 10| Jolie| Diggin|Female|1944-07-31| Nigeria|73.0|
 11| Devlen| Devil| Male|1955-01-11|United States|63.0|
 12| Sibel| Lewisham|Female|1982-11-05| France|35.0|
 13| Paton| Brundrett| Male|1951-11-08| China|66.0|
 14| Cody|Menichillo| Male|1958-08-11| China|59.0|
 15| Brinna| Lannon|Female|1985-07-23| Brazil|32.0|
 16| Abe| Grima| Male|1971-08-23| France|46.0|
 17| Ginger| Lipp| Male|1963-07-16| Japan|54.0|
 18| Braden| Jachimiak| Male|1996-05-10| China|22.0|
 19| Sharl| Airy|Female|1941-09-08| Brazil|76.0|
 20| Erv| Waumsley| Male|1960-07-21| China|57.0|
+-------+----------+----------+------+----------+-------------+----+
only showing top 20 rows

root
-- user_id: integer (nullable = true)
-- first_name: string (nullable = true)
-- last_name: string (nullable = true)
-- gender: string (nullable = true)
-- birth_date: date (nullable = true)
-- country: string (nullable = true)
-- age: double (nullable = true)

Another useful information to have is the age **range** of each user. Using the `when` function, create the following 5 age ranges:
- "(0, 25]"
- "(25, 35]"
- "(35, 45]"
- "(45, 55]"
- "(55, ~]"

And add a new column to the users DataFrame, containing this information.

**Note:** When building logical operations with Spark DataFrames, it's better to be add parantheses. Example:
```python
df.select("name", "age").where( (df("age") > 20) & (df("age") <= 30) )
```

**Note 2:** If you are having problems with the `when` function, you can make an User Defined Function and do your logic in standard python.

In [0]:
ageRank = fn.when((users_from_file.age > 0) & (users_from_file.age <= 25.0), '(0,25])') \
            .when((users_from_file.age > 25) & (users_from_file.age <= 35), '(25,35])') \
            .when((users_from_file.age > 35) & (users_from_file.age <= 45), '(35,45])') \
            .when((users_from_file.age > 45) & (users_from_file.age <= 55), '(45,55])') \
            .otherwise('(55,~])')
users_from_file = users_from_file.withColumn("age_range", ageRank)
users_from_file.show()

+-------+----------+----------+------+----------+-------------+----+---------+
user_id|first_name| last_name|gender|birth_date| country| age|age_range|
+-------+----------+----------+------+----------+-------------+----+---------+
 1| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])|
 2| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])|
 3| Omar| Spinozzi| Male|1977-08-28| China|40.0| (35,45])|
 4| Mag| Kennedy|Female|1963-11-11| China|54.0| (45,55])|
 5| Gerome| Lushey| Male|1990-06-08| Brazil|28.0| (25,35])|
 6| Lazarus| Andrus| Male|1983-04-22| France|35.0| (25,35])|
 7| Adora| Bartunek|Female|1971-06-04| France|47.0| (45,55])|
 8| Gladys| Gulland|Female|1959-07-07| Brazil|58.0| (55,~])|
 9| Pearline| Kitcher|Female|1944-06-01| China|74.0| (55,~])|
 10| Jolie| Diggin|Female|1944-07-31| Nigeria|73.0| (55,~])|
 11| Devlen| Devil| Male|1955-01-11|United States|63.0| (55,~])|
 12| Sibel| Lewisham|Female|1982-11-05| France|35.0| (25,35])|
 13| Paton| Brundrett| Male|1951-11-08| China|66.0| (55,~])|
 14| Cody|Menichillo| Male|1958-08-11| China|59.0| (55,~])|
 15| Brinna| Lannon|Female|1985-07-23| Brazil|32.0| (25,35])|
 16| Abe| Grima| Male|1971-08-23| France|46.0| (45,55])|
 17| Ginger| Lipp| Male|1963-07-16| Japan|54.0| (45,55])|
 18| Braden| Jachimiak| Male|1996-05-10| China|22.0| (0,25])|
 19| Sharl| Airy|Female|1941-09-08| Brazil|76.0| (55,~])|
 20| Erv| Waumsley| Male|1960-07-21| China|57.0| (55,~])|
+-------+----------+----------+------+----------+-------------+----+---------+
only showing top 20 rows

Now that we have improved our users' DataFrame, the first analysis we want to make is the average donation performed by each gender. However, the gender information and the donation value are in different tables, so first we need to join them, using the `user_id` as joining key.

**Note:** Make sure you are not using the `users` DataFrame from the first part of the lab.

**Note 2:** For better performance, you can [broadcast](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.broadcast) the smaller dataset. But you should only do this when you are sure the DataFrame fits in the memory of the nodes.

**J'utilise broadcast sur le petit ensemble qui est `users_from_file`**

In [0]:
#### Sans broadcast
# joined_df = users_from_file.join(donations, 'user_id')

#### Avec Broadcast
joined_df = donations.join(fn.broadcast(users_from_file), 'user_id')

display(joined_df)

user_id,donation_id,donation_value,date,time,timestamp,first_name,last_name,gender,birth_date,country,age,age_range
63,1,25.41,2017-03-06,18:28:20,2017-03-06T18:28:20.000+0000,Tabor,Bramer,Male,1983-05-29,Brazil,35.0,"(25,35])"
53,2,93.32,2017-03-15,15:15:09,2017-03-15T15:15:09.000+0000,Chris,Climie,Male,1970-06-02,France,48.0,"(45,55])"
74,3,136.04,2017-10-29,20:36:23,2017-10-29T20:36:23.000+0000,Livia,Lodemann,Female,1967-03-11,United States,51.0,"(45,55])"
94,4,37.25,2017-05-16,12:33:58,2017-05-16T12:33:58.000+0000,Kendre,Pankhurst.,Female,1941-05-28,China,77.0,"(55,~])"
83,5,89.45,2017-05-09,22:45:44,2017-05-09T22:45:44.000+0000,Quincy,Marquess,Male,1997-08-13,Brazil,20.0,"(0,25])"
80,6,163.67,2017-07-25,21:50:36,2017-07-25T21:50:36.000+0000,Boothe,Endley,Male,1960-03-03,China,58.0,"(55,~])"
19,7,10.69,2017-11-15,8:43:54,2017-11-15T08:43:54.000+0000,Sharl,Airy,Female,1941-09-08,Brazil,76.0,"(55,~])"
53,8,135.09,2017-04-14,7:43:37,2017-04-14T07:43:37.000+0000,Chris,Climie,Male,1970-06-02,France,48.0,"(45,55])"
46,9,94.38,2017-03-07,10:30:34,2017-03-07T10:30:34.000+0000,Ethelyn,Raubenheim,Female,1953-07-19,Brazil,64.0,"(55,~])"
90,10,68.63,2017-02-02,14:23:43,2017-02-02T14:23:43.000+0000,Farlee,O'Connell,Male,1985-05-15,China,33.0,"(25,35])"


Now, use aggregation to find the the min, max and avg donation by gender.

In [0]:
donations_by_gender = joined_df.groupBy('gender').agg(fn.min('donation_value').alias('min'),
                                                      fn.max('donation_value').alias('max'),
                                                      fn.avg('donation_value').alias("avg")
                                                      )
donations_by_gender.show()

+------+----+------+------------------+
gender| min| max| avg|
+------+----+------+------------------+
Female|1.25|199.87| 97.62667400881057|
 Male|1.12|199.93|100.23606227106244|
+------+----+------+------------------+

Now, make the necessary transformations and aggregations to answer to the following questions about the data. Note that some questions are only about the users, so make sure to use the smaller possible dataset when looking for your answers!

**Question 1:** 

a) What's the average, min and max age of the users? 

b) What's the average, min and max age of the users, by gender?

In [0]:
result_1a = users_from_file.agg(fn.avg('age').alias('average'),
                                fn.min('age').alias('min'),
                                fn.max('age').alias('max')
                               )
result_1a.show()

result_1b = users_from_file.groupBy('gender').agg(fn.avg('age').alias('average'),
                                                  fn.min('age').alias('min'),
                                                  fn.max('age').alias('max')
                                                 )
result_1b.show()

+-------+----+----+
average| min| max|
+-------+----+----+
 48.68|19.0|78.0|
+-------+----+----+

+------+------------------+----+----+
gender| average| min| max|
+------+------------------+----+----+
Female| 52.97959183673469|19.0|78.0|
 Male|44.549019607843135|19.0|76.0|
+------+------------------+----+----+

**Question 2:**

a) How many distinct country origins exist in the data? Print a DataFrame listing them.

b) What are the top 5 countries with the most users? Print a DataFrame containing the name of the countries and the counts.

In [0]:
result_2a = users_from_file.select('country').distinct()
result_2a.show()

result_2b = users_from_file.groupBy('country').agg(fn.count('user_id').alias('count_country')).orderBy('count_country', ascending=False).limit(5)
result_2b.show()

+-------------+
 country|
+-------------+
 Germany|
 France|
United States|
 China|
 Nigeria|
 Brazil|
 Japan|
 New Zealand|
+-------------+

+-------------+-------------+
 country|count_country|
+-------------+-------------+
 China| 54|
 Brazil| 19|
 France| 13|
United States| 5|
 Japan| 4|
+-------------+-------------+

**Question 3:**

What's the number of donations average, min and max donations values by age range?

In [0]:
result_3 = joined_df.groupBy('age_range').agg( fn.count('donation_id').alias('number of donations').alias('number_of_donations'),
                                               fn.sum('donation_value').alias('sum_of_donations_value'),
                                               fn.min('donation_value').alias('min'),
                                               fn.max('donation_value').alias('max'),
                                               fn.avg('donation_value').alias("avg")
                                              )
result_3.show()

+---------+-------------------+----------------------+----+------+-----------------+
age_range|number_of_donations|sum_of_donations_value| min| max| avg|
+---------+-------------------+----------------------+----+------+-----------------+
 (25,35])| 238| 22748.540000000005|1.12|199.87|95.58210084033615|
 (55,~])| 337| 33578.369999999995|1.36|197.79|99.63908011869435|
 (45,55])| 206| 21122.890000000003|1.19|199.93|102.5383009708738|
 (35,45])| 139| 13651.069999999989|2.03|199.32| 98.2091366906474|
 (0,25])| 80| 7950.5300000000025|4.33|199.28|99.38162500000003|
+---------+-------------------+----------------------+----+------+-----------------+

**Question 4:**

a) What's the number of donations, average, min and max donation values by user location (country)?

b) What is the number of donations, average, min and max donation values by gender for each user location (contry)? (the resulting DataFrame must contain 5 columns: the gender of the user, their country and the 3 metrics)

In [0]:
result_4a = joined_df.groupBy('country').agg(  fn.count('donation_id').alias('number of donations').alias('number_of_donations'),
                                               fn.sum('donation_value').alias('sum_of_donations_value'),
                                               fn.avg('donation_value').alias("avg"),
                                               fn.min('donation_value').alias('min'),
                                               fn.max('donation_value').alias('max'),
                                              )
result_4a.show()

result_4b = joined_df.groupBy(['gender', 'country']).agg(  
#                                                 fn.count('donation_id').alias('number of donations').alias('number_of_donations'),
                                               fn.avg('donation_value').alias("avg"),
                                               fn.min('donation_value').alias('min'),
                                               fn.max('donation_value').alias('max'),
                                              )
result_4b.show()

+-------------+-------------------+----------------------+------------------+-----+------+
 country|number_of_donations|sum_of_donations_value| avg| min| max|
+-------------+-------------------+----------------------+------------------+-----+------+
 Germany| 19| 1817.1899999999998| 95.64157894736842|17.81|180.48|
 France| 120| 11475.839999999998| 95.63199999999999| 5.79|199.93|
United States| 50| 5418.98| 108.3796| 1.36|198.15|
 China| 563| 54962.050000000054| 97.62353463587931| 1.25|199.32|
 Nigeria| 20| 1624.5200000000002| 81.22600000000001|14.47|192.63|
 Brazil| 180| 19027.180000000022|105.70655555555568| 1.12|199.91|
 Japan| 37| 3538.4100000000008| 95.63270270270273| 1.19|199.76|
 New Zealand| 11| 1187.23| 107.93|12.98|172.51|
+-------------+-------------------+----------------------+------------------+-----+------+

+------+-------------+------------------+-----+------+
gender| country| avg| min| max|
+------+-------------+------------------+-----+------+
Female| Japan|116.16818181818181|23.52|192.32|
 Male| France| 95.74000000000001| 5.79|199.93|
 Male| Germany| 95.64157894736842|17.81|180.48|
Female|United States|124.63000000000001|26.15|195.98|
Female| New Zealand| 107.93|12.98|172.51|
Female| Nigeria| 81.22600000000001|14.47|192.63|
 Male|United States| 89.30304347826086| 1.36|198.15|
 Male| Japan| 86.94461538461542| 1.19|199.76|
Female| Brazil|102.05333333333331| 1.49|196.41|
 Male| China| 100.5750485436893| 1.47|199.32|
Female| China| 94.0329133858268| 1.25|198.88|
 Male| Brazil|108.14203703703706| 1.12|199.91|
Female| France| 95.52033898305086|14.36|199.87|
+------+-------------+------------------+-----+------+

**Question 5**

Which month of the year has the largest aggregated donation value?

**Hint**: you can use a function to extract the month from a date, then you can aggregate to find the total donation value.

In [0]:
result_5 = donations.withColumn("month", fn.month(donations.date)) \
                    .groupBy("month") \
                    .agg(fn.sum("donation_value").alias("sum_of_donations")) \
                    .orderBy("sum_of_donations", ascending = False) \
                    .limit(1)
result_5.show()

+-----+------------------+
month| sum_of_donations|
+-----+------------------+
 5|10696.400000000001|
+-----+------------------+

### 3. Window Functions

_This section uses the same data as the last one._

Window functions are very useful for gathering aggregated data without actually aggregating the DataFrame. They can also be used to find "previous" and "next" information for entities, such as an user. [This article](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html) has a very nice explanation about the concept.

We want to find the users who donated less than a threshold and remove them from the donations dataset. Now, there are two ways of doing that:

1) Performing a traditional aggregation to find the users who donated less than the threshold, then filtering these users from the donations dataset, either with `where(not(df("user_id").isin(a_local_list)))` or with a join of type "anti-join".

2) Using window functions to add a new column with the aggregated donations per user, and then using a normal filter such as `where(aggregated_donation < threshold)`

Let's implement both and compare the complexity:

First, perform the traditional aggregation and find the users who donated less than 500 in total:

In [0]:
bad_users = joined_df.groupBy("user_id") \
                     .agg(fn.sum("donation_value").alias("somme_donation")) \
                     .where("somme_donation< 500")
bad_users.show()

+-------+------------------+
user_id| somme_donation|
+-------+------------------+
 85|437.78999999999996|
 47| 351.84|
 96| 354.97|
 84|329.46999999999997|
 87|497.60999999999996|
 82| 291.73|
 73|455.20000000000005|
 25|413.09000000000003|
 42| 215.35|
 99| 370.99|
+-------+------------------+

You should have found around 10 users. Now, perform an "anti-join" to remove those users from `joined_df`.

**Hint:** The `join` operation accepts a third argument which is the join type. The accepted values are: 'inner', 'outer', 'full', 'fullouter', 'leftouter', 'left', 'rightouter', 'right', 'leftsemi', 'leftanti', 'cross'.

In [0]:
good_donations = joined_df.join(bad_users, "user_id", "leftanti")
good_donations.count()

Out[132]: 955

Verify if the count of `good_donations` makes sense by performing a normal join to find the `bad_donations`.

In [0]:
bad_donations = joined_df.join(bad_users, "user_id")
bad_donations.count()

Out[133]: 45

If you done everything right, at this point `good_donations.count()` + `bad_donations.count()` = `joined_df.count()`.

But using the join approach can be very heavy and it requires multipe operations. For this kind of problems, Window Functions are better.

In [0]:
good_donations.count() + bad_donations.count() == joined_df.count()

Out[134]: True

The first step is to create your window specification over `user_id` by using partitionBy

In [0]:
from pyspark.sql import Window

window_spec = Window.partitionBy('user_id')

Then, you can use one of the window functions of the `pyspark.sql.functions` package, appling to the created window_spec by using the `over` method. 

**Hint:** If you are blocked, try looking at the [documentation](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.over) for the `over` method, or searching on StackOverflow.

In [0]:
new_column = fn.sum("donation_value").over(window_spec)

donations_with_total = joined_df.withColumn('total_donated_by_user', new_column)
donations_with_total.show()

+-------+-----------+--------------+----------+--------+-------------------+----------+---------+------+----------+-------------+----+---------+---------------------+
user_id|donation_id|donation_value| date| time| timestamp|first_name|last_name|gender|birth_date| country| age|age_range|total_donated_by_user|
+-------+-----------+--------------+----------+--------+-------------------+----------+---------+------+----------+-------------+----+---------+---------------------+
 1| 145| 95.88|2017-05-13| 5:27:25|2017-05-13 05:27:25| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])| 889.16|
 1| 264| 150.51|2017-09-21| 4:36:37|2017-09-21 04:36:37| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])| 889.16|
 1| 317| 51.55|2017-10-26| 1:55:37|2017-10-26 01:55:37| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])| 889.16|
 1| 486| 188.37|2017-02-01|20:20:44|2017-02-01 20:20:44| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])| 889.16|
 1| 560| 121.82|2016-12-03| 3:34:03|2016-12-03 03:34:03| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])| 889.16|
 1| 561| 195.98|2017-09-01| 4:04:17|2017-09-01 04:04:17| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])| 889.16|
 1| 564| 85.05|2017-08-11| 0:19:06|2017-08-11 00:19:06| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])| 889.16|
 2| 36| 197.01|2016-11-28|15:41:01|2016-11-28 15:41:01| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])| 1297.1200000000001|
 2| 54| 118.02|2017-05-22| 9:40:27|2017-05-22 09:40:27| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])| 1297.1200000000001|
 2| 138| 173.28|2017-06-29|10:03:02|2017-06-29 10:03:02| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])| 1297.1200000000001|
 2| 222| 154.52|2017-09-10| 5:54:41|2017-09-10 05:54:41| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])| 1297.1200000000001|
 2| 316| 190.49|2017-05-16| 8:41:48|2017-05-16 08:41:48| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])| 1297.1200000000001|
 2| 460| 65.68|2017-05-11| 3:32:57|2017-05-11 03:32:57| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])| 1297.1200000000001|
 2| 524| 91.76|2017-05-07|14:32:42|2017-05-07 14:32:42| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])| 1297.1200000000001|
 2| 628| 172.69|2017-09-09|21:20:39|2017-09-09 21:20:39| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])| 1297.1200000000001|
 2| 642| 133.67|2017-08-26|18:39:30|2017-08-26 18:39:30| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])| 1297.1200000000001|
 3| 122| 6.77|2017-10-12| 5:18:38|2017-10-12 05:18:38| Omar| Spinozzi| Male|1977-08-28| China|40.0| (35,45])| 1143.81|
 3| 177| 152.98|2016-12-31|21:14:54|2016-12-31 21:14:54| Omar| Spinozzi| Male|1977-08-28| China|40.0| (35,45])| 1143.81|
 3| 268| 58.17|2017-01-21| 8:42:12|2017-01-21 08:42:12| Omar| Spinozzi| Male|1977-08-28| China|40.0| (35,45])| 1143.81|
 3| 325| 17.3|2017-09-23|14:47:50|2017-09-23 14:47:50| Omar| Spinozzi| Male|1977-08-28| China|40.0| (35,45])| 1143.81|
+-------+-----------+--------------+----------+--------+-------------------+----------+---------+------+----------+-------------+----+---------+---------------------+
only showing top 20 rows

And now you can just filter on the `total_donated_by_user` column:

In [0]:
good_donations_wf = donations_with_total.where("total_donated_by_user >= 500")
good_donations_wf.count()

Out[138]: 955

If you done everything right, you should obtain `good_donations_wf.count()` = `good_donations.count()`

In [0]:
good_donations_wf.count() == good_donations.count()

Out[139]: True

Window functions also can be useful to find the "next" and "previous" operations by using `functions.lead` and `functions.lag`. In our example, we can use it to find the date interval between two donations by a specific user.

For this kind of Window function, the window specification must be ordered by the date. So, create a new window specification partitioned by the `user_id` and ordered by the donation timestamp.

In [0]:
ordered_window = Window.partitionBy('user_id').orderBy("timestamp")

Now use `functions.lag().over()` to add a column with the timestamp of the previous donation of the same user. Then, inspect the result to see what it looks like.

In [0]:
new_column = fn.lag("timestamp", 1).over(ordered_window)

donations_with_lag = good_donations.withColumn('timestamp_order_lag', new_column)
donations_with_lag.orderBy("user_id", "timestamp").show()

+-------+-----------+--------------+----------+--------+-------------------+----------+---------+------+----------+-------------+----+---------+-------------------+
user_id|donation_id|donation_value| date| time| timestamp|first_name|last_name|gender|birth_date| country| age|age_range|timestamp_order_lag|
+-------+-----------+--------------+----------+--------+-------------------+----------+---------+------+----------+-------------+----+---------+-------------------+
 1| 560| 121.82|2016-12-03| 3:34:03|2016-12-03 03:34:03| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])| null|
 1| 486| 188.37|2017-02-01|20:20:44|2017-02-01 20:20:44| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])|2016-12-03 03:34:03|
 1| 145| 95.88|2017-05-13| 5:27:25|2017-05-13 05:27:25| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])|2017-02-01 20:20:44|
 1| 564| 85.05|2017-08-11| 0:19:06|2017-08-11 00:19:06| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])|2017-05-13 05:27:25|
 1| 561| 195.98|2017-09-01| 4:04:17|2017-09-01 04:04:17| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])|2017-08-11 00:19:06|
 1| 264| 150.51|2017-09-21| 4:36:37|2017-09-21 04:36:37| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])|2017-09-01 04:04:17|
 1| 317| 51.55|2017-10-26| 1:55:37|2017-10-26 01:55:37| Janifer| Lagadu|Female|1940-08-09|United States|77.0| (55,~])|2017-09-21 04:36:37|
 2| 36| 197.01|2016-11-28|15:41:01|2016-11-28 15:41:01| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])| null|
 2| 524| 91.76|2017-05-07|14:32:42|2017-05-07 14:32:42| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])|2016-11-28 15:41:01|
 2| 460| 65.68|2017-05-11| 3:32:57|2017-05-11 03:32:57| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])|2017-05-07 14:32:42|
 2| 316| 190.49|2017-05-16| 8:41:48|2017-05-16 08:41:48| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])|2017-05-11 03:32:57|
 2| 54| 118.02|2017-05-22| 9:40:27|2017-05-22 09:40:27| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])|2017-05-16 08:41:48|
 2| 138| 173.28|2017-06-29|10:03:02|2017-06-29 10:03:02| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])|2017-05-22 09:40:27|
 2| 642| 133.67|2017-08-26|18:39:30|2017-08-26 18:39:30| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])|2017-06-29 10:03:02|
 2| 628| 172.69|2017-09-09|21:20:39|2017-09-09 21:20:39| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])|2017-08-26 18:39:30|
 2| 222| 154.52|2017-09-10| 5:54:41|2017-09-10 05:54:41| Bruis| Danilov| Male|1966-05-28| Brazil|52.0| (45,55])|2017-09-09 21:20:39|
 3| 566| 125.9|2016-12-01|13:49:19|2016-12-01 13:49:19| Omar| Spinozzi| Male|1977-08-28| China|40.0| (35,45])| null|
 3| 177| 152.98|2016-12-31|21:14:54|2016-12-31 21:14:54| Omar| Spinozzi| Male|1977-08-28| China|40.0| (35,45])|2016-12-01 13:49:19|
 3| 268| 58.17|2017-01-21| 8:42:12|2017-01-21 08:42:12| Omar| Spinozzi| Male|1977-08-28| China|40.0| (35,45])|2016-12-31 21:14:54|
 3| 387| 74.61|2017-02-17| 3:04:01|2017-02-17 03:04:01| Omar| Spinozzi| Male|1977-08-28| China|40.0| (35,45])|2017-01-21 08:42:12|
+-------+-----------+--------------+----------+--------+-------------------+----------+---------+------+----------+-------------+----+---------+-------------------+
only showing top 20 rows

Finally, compute the average time it took for each user between two of their consecutive donations (in days), and print the 5 users with the smallest averages. The result must include at least the users' id, last name and birth date, as well as the computed average.

**J'applique join après groupBy pour récupérer pour chaque user: first_name, last_name, ...**

In [0]:
average_between_donation = fn.avg(fn.datediff("timestamp", "timestamp_order_lag"))

users_average_between_donations = donations_with_lag.groupBy("user_id").agg(average_between_donation.alias("average_between_donation")) \
                                                    .join(users_from_file, 'user_id') \
                                                    .orderBy("average_between_donation")

users_average_between_donations.show(5)

+-------+------------------------+-----------+---------+------+----------+-------+----+---------+
user_id|average_between_donation| first_name|last_name|gender|birth_date|country| age|age_range|
+-------+------------------------+-----------+---------+------+----------+-------+----+---------+
 67| 14.23076923076923| Zollie| Hambatch| Male|1954-04-14| Brazil|64.0| (55,~])|
 78| 16.0|Christoforo| Sherston| Male|1965-07-23| France|52.0| (45,55])|
 91| 17.88888888888889| Salem| Kildale| Male|1979-01-25|Germany|39.0| (35,45])|
 6| 18.529411764705884| Lazarus| Andrus| Male|1983-04-22| France|35.0| (25,35])|
 63| 18.88888888888889| Tabor| Bramer| Male|1983-05-29| Brazil|35.0| (25,35])|
+-------+------------------------+-----------+---------+------+----------+-------+----+---------+
only showing top 5 rows

Congratulations, you have finished this notebook!